In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="E:\Workspace\Python\Workspace\ML_AI\singular-server-393911-b92f5a618cf9.json"

In [2]:
from google.cloud import bigquery
client = bigquery.Client()

query = """
SELECT
  date, SUM(confirmed) num_reports
FROM `bigquery-public-data.covid19_open_data.compatibility_view`
WHERE country_region = '{country}'
    AND date BETWEEN '{start_date}' AND '{end_date}'
GROUP BY date
HAVING num_reports IS NOT NULL
ORDER BY date ASC
"""

#function to estimate the query output's size in bytes
def estimate_query_size(query, parameters):
    if parameters:
        query = query.format(**parameters)    
    job_config = bigquery.job.QueryJobConfig(dry_run=True)
    job = client.query(query, job_config=job_config)
    return f"Estimated Query size will be: {job.total_bytes_processed} bytes"


def execute_query(query, parameters):
    
    # Replacing parameter values
    if parameters:
        query = query.format(**parameters)

    query_job = client.query(query)
    results = query_job.result()

    if results.total_rows == 0:
        print("Query returns an empty table")
        return None

    print("Query returns a valid table")
    return results


country = input("Enter a country name:").strip()
start_date = input("Enter the start date (YYYY-MM-DD):").strip()
end_date = input("Enter the end date (YYYY-MM-DD):").strip()
df_name = input("Enter the DataFrame name:").strip()


parameters = {
    "country": country,
    "start_date": start_date,
    "end_date": end_date
}


print(estimate_query_size(query,parameters))
results = execute_query(query, parameters)

if results:
    globals()[df_name] = results.to_dataframe()
    print(f"{df_name} DataFrame created successfully.")





ModuleNotFoundError: No module named 'google.cloud'

**EDA on covid dataset**